<a href="https://colab.research.google.com/github/rkurniawati/pyjama-patternlets/blob/master/Java_OpenMP_Patternlets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Java OpenMP Patternlet Notebook

Adapted to Java by Ruth Kurniawati (Westfield State University) based on the [PDC book](https://pdcbook.calvin.edu/pdcbook/RaspberryPiHandout/) from [CSInParallel](https://csinparallel.org/index.html). 

This notebook contains OpenMP patternlet examples in the Java programming language. Patterns are reusable solution for commonly occuring problems. The OpenMP patternlets are reusable solution that were originally written in the C language with the OpenMP library by Joel Adams: 

> Adams, Joel C. "Patternlets: A Teaching Tool for Introducing Students to Parallel Design Patterns." 2015 IEEE International Parallel and Distributed Processing Symposium Workshop. IEEE, 2015.

However, OpenMP library is only available for C/C++ and Fortran languages. For Java, Pyjama provides support for OpenMP-like directive. More information about Pyjama can be found in the paper below:

> Vikas, Nasser Giacaman, and Oliver Sinnen. 2013. Pyjama: OpenMP-like implementation for Java, with GUI extensions. In <i>Proceedings of the 2013 International Workshop on Programming Models and Applications for Multicores and Manycores</i> (<i>PMAM '13</i>). Association for Computing Machinery, New York, NY, USA, 43–52. DOI:https://doi.org/10.1145/2442992.2442997

# Multicore Systems and Multi-Threading

Before proceeding with the examples, let's investigate the computer that this notebook is running on. For this, let's use the `lscpu` command. 




In [ ]:
!lscpu

## Cores, Processes and Threads



If you run `lscpu` in the notebook, you may see output similar to below:

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
```

This output means that the computer has 2 CPUs -- however, there is actually only one physical core but this core can execute 2 execution threads. 

The `lscpu` command tells us a LOT of useful information, including the number of available cores. In this case we know that there is one socket (or chip) with 1 physical core, where each core can support 2 thread. On larger systems, it is common to see multiple threads supported per core. This is an example of **simultaneous multi-threading** (SMT, or Hyperthreading on Intel systems). A **core** can be thought of as the compute unit of the CPU. It includes registers, an ALU, and control units.

Before we can discuss what a thread is, we must first discuss what a process is. A **process** can be thought of as an abstraction of a running program. When you type a command into the command line and press Enter, the Bash shell launches a process associated with that program executable. Each process contains a copy of the code and data of the program executable, and its own allocation of the stack and heap.

A **thread** is a light-weight process. While each thread gets its own stack allocation, it shares the heap, code and data of the parent process. As a result, all the threads in a multi-threaded process can access a common pool of memory. This is why multi-threading is commonly referred to as shared memory programming. A single-threaded process is also referred to as a serial process or program.

### Process Execution
A multicore CPU allows multiple processes to execute simultaneously, or in **parallel**. While the terms concurrency and parallel are related, it is useful to think of concurrency as a software/OS-level concept, while parallel as a hardware/execution concept. A multi-threaded program, while capable of parallel execution, runs concurrently on a system with only a single CPU core.

## Thread Execution

The primary goal of creating multi-threaded programs is to decrease the speed of a program’s execution. In a program that is perfectly parallelizable (that is, all components are paralleizable), it is usually possible to distribute the work associated with a program equally among all the threads. For a program _p_ whose work is equally distributed among _t_ threads, it will take roughly _p_/_t_ time, if executed on _t_ cores.

For example, if a multi-threaded process that is perfectly parallelized takes 100 seconds to execute on one core, on a multi-core system with 4 cores the program will take approximately 100/4 seconds = 25 seconds to execute.

## Leveraging Multiple Cores
While multicore processors are ubiquitous in today’s world, most of the popular programming languages were designed to support single-thread execution. However, several native libraries are available for supporting multi-threading in popular languages like C/C++ and FORTRAN.

One of these libraries is the Open MultiProcessing (OpenMP), a popular API for shared memory programming, and a standard since 1997. A key benefit of OpenMP over explicit threading libraries like POSIX threads is the ability to incrementally add parallelism to a program. For standard threaded programs, it is usually necessary to write a lot of extra code to add multi-threading to a program. Instead, OpenMP employs a series of pragmas, or special compiler directives, that tell the compiler how to parallelize the code.

OpenMP library is only available for C/C++ and Fortran languages. For Java, Pyjama compiler and runtime provide support for OpenMP-like directive. More information about Pyjama can be found in the paper below:

Vikas, Nasser Giacaman, and Oliver Sinnen. 2013. Pyjama: OpenMP-like implementation for Java, with GUI extensions. In <i>Proceedings of the 2013 International Workshop on Programming Models and Applications for Multicores and Manycores</i> (<i>PMAM '13</i>). Association for Computing Machinery, New York, NY, USA, 43–52. DOI:https://doi.org/10.1145/2442992.2442997

In the rest of this notebook, we will look at Java programs that use OpenMP-like directives provided by the Pyjama to explore small patterns (_patternlets_) in parallel programming. 

# Pyjama OpenMP library

Before proceeding to the code sample, we need to make Pyjama compiler and runtime available to this notebook. 

The Pyjama library used in this notebook is obtained from the [CDER project](https://tcpp.cs.gsu.edu/curriculum/?q=node/21183), specifically [the version with additional bug fixes provided by Tennnessee Tech](https://www.csc.tntech.edu/pdcincs/index.php/installation). 

## Pyjama setup

First, let's download and setup Pyjama Java source code compiler and runtime library. The commands below will download a ZIP file from Tennessee Tech, unzip it, and create the Pyjama/Pyjama.jar link to point to the specific jar file extracted from the ZIP file.

In [ ]:
!wget -O Pyjama.zip https://www.csc.tntech.edu/pdcincs/resources/modules/tools/updated/Pyjama.zip
!unzip -o -d lib Pyjama.zip 
!ln lib/Pyjama/Pyjama-3.1.0.jar lib/Pyjama.jar

## Hello world

In this example, we will verify that the Pyjama installation is working and able to create multiple threads as specified in the `#omp parallel num_threads` directive.

In [ ]:
%%writefile HelloWorld.java
public class HelloWorld
{
	
	public static void main(String[] args) 
	{
 
    Pyjama.omp_set_num_threads(10);
		//#omp parallel
		{
			int id = Pyjama.omp_get_thread_num();
      int numThreads = Pyjama.omp_get_num_threads();
      System.out.print("Hello from thread " + id+ ", ");
      System.out.println("of a total of "+ numThreads+ " threads.");
		}
	}
}

Note that the OpenMP directive is specified inside a single line comment that starts with `//`. For the directive to be recognized by Pyjama compiler, the `//` has to be followed immediately by `#omp`. 

First, let's use the Pyjama compiler to process the `#omp parallel` directive in the program.

In [ ]:
!java -jar lib/Pyjama.jar HelloWorld.java

Now, we're ready to run the HelloWorld program. To do this, you will need to make the Pyjama.jar available in the classpath so that the Pyjama OpenMP-like runtime library is available to the HelloWorld program.

In [ ]:
!java -cp lib/Pyjama.jar:. HelloWorld

You should expect 10 lines of `Hello from thread x, of a total of y threads`. The lines may be interspersed with one another if you have more than one processors running the code. An example output is below.

```
Hello from thread 9, Hello from thread 5, Hello from thread 8, Hello from thread 2, of a total of 10 threads.
of a total of 10 threads.
Hello from thread 6, Hello from thread 3, of a total of 10 threads.
Hello from thread 4, of a total of 10 threads.
of a total of 10 threads.
of a total of 10 threads.
Hello from thread 7, of a total of 10 threads.
Hello from thread 0, of a total of 10 threads.
Hello from thread 1, of a total of 10 threads.
of a total of 10 threads.
```

# A Simple Parallel Program

## The SPMD Patternlet


A patternlet is a small program that succinctly illustrates common patterns in parallel programming. The first patternlet we will study is Single Program, Multiple Data (SPMD). Let’s start by examining Spmd2.java, a program that uses OpenMP pragmas to make it easy to run a portion of the program on multiple threads. Note that the variables `id` and `numThreads` are shared among the threads.

In [ ]:
%%writefile Spmd2.java
class Spmd2 {
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        int id, numThreads;
        //#omp parallel shared(id, numThreads)
        {
            numThreads = Pyjama.omp_get_num_threads();
            id = Pyjama.omp_get_thread_num();
            System.out.println("Hello from thread "+ id +" of " + numThreads);
        }

        System.out.println();
    }
}


The `omp parallel` directive tells the Pyjama compiler that the block of code within the curly braces be run on separate threads. Prior to the line with the directive, the program is run serially. When block of code marked with the `omp parallel` directive executes, Pyjama generates a a team of threads (known as forking). Each thread is assigned its own id and runs separate copies of the code between the curly braces. At the end of the block scope, Pyjama combines all the threads together to a single-threaded process (known as joining). Conceptually, the process looks like the following.

<img src=https://pdcbook.calvin.edu/pdcbook/RaspberryPiHandout/_images/ForkJoin_SPMD.png >

## Running the Program

Just like in the HelloWorld example, first we need to use the Pyjama compiler to process the `#omp` directive. 

In [ ]:
!java -jar lib/Pyjama.jar Spmd2.java

Now, we're ready to run the Spmd2 program. Let's specify that you'd like to have 10 threads by supplying this number in the command line argument. 

In [ ]:
!java -cp lib/Pyjama.jar:. Spmd2 10 

Try running the program a few times with 10 threads (press the run button in the cell above). Observe the output. Occasionally something will be amiss. Do you notice it?

## Race Conditions

Watch this [video](
https://d32ogoqmya1dw8.cloudfront.net/files/csinparallel/raceconditions_workshop.mov) to help you understand what's going on. Note that the video is made for the C++ version of the program, however the underlying issue is the same. 

The Spmd2 program has a race condition where there are more than one threads modifying a shared variable. Which shared variable(s) is/are causing the problem?

## Fixing the code

For this example, the race condition can be avoided by ensuring that each threads has its own copy of `id` and `numThreads` variables. Instead of declaring them as `shared` in the `omp parallel` directive, use the `private` clause as shown below.

In [ ]:
%%writefile Spmd2.java
class Spmd2 {
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        int id, numThreads;
        //#omp parallel private(id, numThreads)
        {
            numThreads = Pyjama.omp_get_num_threads();
            id = Pyjama.omp_get_thread_num();
            System.out.println("Hello from thread "+ id +" of " + numThreads);
        }

        System.out.println();
    }
}


Let's compile and run this modified program.

In [ ]:
!java -jar lib/Pyjama.jar Spmd2.java

In [ ]:
!java -cp lib/Pyjama.jar:. Spmd2 10 

Were you able to reproduce the race condition using the corrected program? Why should you also declare `numThreads` as a private variable?

# Running Loops in Parallel

Next we will consider a program that has a loop in it. An iterative for loop is a remarkably common pattern in all programming, primarily used to perform a calculation N times, often over a set of data containing N elements, using each element in turn inside the for loop.

If there are no dependencies between the iterations (i.e. the order of them is not important), then the code inside the loop can be split between forked threads. However, the programmer must first decide how to partition the work between the threads. Specifically, how many and which iterations of the loop will each thread complete on its own?

The **data decomposition** pattern describes the way how work is distributed across multiple threads. This chapter presents two patternlets, parallelLoop-equalChunks and parallelLoop-chunksOf1, that describe two common data decomposition strategies.

## Parallel Loop, Equal Chunks

Let's experiment with another OpenMP directive that will divide the work in a loop into equal chunks. 

In [ ]:
%%writefile ParallelLoopEqualChunks.java
class ParallelLoopEqualChunks {
    final static int REPS = 16;
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        //#omp parallel for  
        for (int i = 0; i < REPS; i++) {
            int id = Pyjama.omp_get_thread_num();
            System.out.println("Thread "+id+" performed iteration "+i);
        }

        System.out.println();
    }
}


The `omp parallel for` directive tells the Pyjama OpenMP compiler to do the following:
- Generate a team of threads (default is equal to the number of cores)
- Assign each thread an equal number of iterations (a chunk) of the for loop.
- At the end of the scope of the for loop, join all the theads back to a single-threaded process.

As in our previous example, the code up to the `omp parallel for` directive is run serially. The code that is in the scope of the `omp parallel for` directive (everything inside the for loop) is run in parallel, with a subset of iterations assigned to each thread. After the implicit join at the end of the for loop, the program once again is a single-threaded process that executes serially to completion.

In the above program, REPS is set to 16. If the program is run with 4 threads, then each thread gets 4 iterations of the loop (see illustration below):

<img src="https://pdcbook.calvin.edu/pdcbook/RaspberryPiHandout/_images/ParallelFor_Chunks-4_threads-1.png">

## Try It Out

Try compile and run the program using the following commands below.

In [ ]:
!java -jar lib/Pyjama.jar ParallelLoopEqualChunks.java

In [ ]:
!java -cp lib/Pyjama.jar:. ParallelLoopEqualChunks 4

Try running the program a few times with 4 threads. How does the work in the for loop get assigned to the threads?

### Unequal Iterrations

Also try using a different number of threads. Pick a number so that the number iterations cannot be equally divided by the number of threads, such as 5. 

In [ ]:
!java -cp lib/Pyjama.jar:. ParallelLoopEqualChunks 5

What happens to the extra iterations?

This equal-chunk decomposition is especially useful in the following scenarios:
- Each iteration of the loop takes the same amount of time to finish
- The loop involves accesses to data in consecutive memory locations (e.g. an array), allowing the program to take advantage of spatial locality.

## Parallel Loop, Chunks of 1

In some cases, it makes sense to have iterations assigned to threads in “round-robin” style. In other words, iteration 0 goes to thread 0, iteration 1 goes to thread 1, iteration 2 goes to thread 2, and so on.

Let's examine the code below that directs Pyjama to do this.

In [ ]:
%%writefile ParallelLoopChunksOf1.java
class ParallelLoopChunksOf1 {
    final static int REPS = 16;
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        //#omp parallel for schedule(static,1)
        for (int i = 0; i < REPS; i++) {
            int id = Pyjama.omp_get_thread_num();
            System.out.println("Thread "+id+" performed iteration "+i);
        }

        System.out.println();
    }
}

The code is nearly identical to the previous program. The difference is in the `omp` directive. The `omp parallel for` directive has a new `schedule` clause which specifies the way iterations should be assigned to threads. The `static` keyword indicates that the the compiler should assign work to each thread at compile time (a **static** scheduling policy). The `1` indicates that the chunk size should be 1 iteration. Therefore, the above code would have 16 total chunks.

In the case where the number of chunks exceed the number of theads, each successive chunk is assigned to a thread in round-robin fashion.

### Try It Out

Let's compile and run the code. 

In [ ]:
!java -jar lib/Pyjama.jar ParallelLoopChunksOf1.java

In [ ]:
!java -cp lib/Pyjama.jar:. ParallelLoopChunksOf1 4

## Another way to do round-robin scheduling

At this point, you might have guessed that `#omp parallel` creates the threads and `#omp for` divides the work according to the schedule specified (the default is `static` with chunk size equals to the number of iterations divided by the number of threads). With this in mind, we can write code that will do round-robin scheduling of the for loop without using `#omp for`. 



In [ ]:
%%writefile ParallelLoopChunksOf1.java
class ParallelLoopChunksOf1 {
    final static int REPS = 16;
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        //#omp parallel
        {
            int numThreads = Pyjama.omp_get_num_threads();
            int id = Pyjama.omp_get_thread_num();
            for (int i = id; i < REPS; i+=numThreads) {
                System.out.println("Thread "+id+" performed iteration "+i);
            }
        }
        System.out.println();
    }
}



In [ ]:
!java -jar lib/Pyjama.jar ParallelLoopChunksOf1.java

In [ ]:
!java -cp lib/Pyjama.jar:. ParallelLoopChunksOf1 4

Compare the work assignment in this program with the previous program that uses `#omp for` directive.

## Dynamic Scheduling

In some cases, it is beneficial for the assignment of loop iterations to occur at run-time. This is especially useful when each iteration of the loop can take a different amount of time. Dynamic scheduling, or assigning iterations to threads at run time, allows threads that have finished work to start on new work, while letting threads that are still busy continue to work in peace.

An example of program that will benefit from this scheduling is shown below. This program counts the number of prime numbers between 1 and n. The amount of work involved in checking if a number is a prime number depends on the value of the number -- larger number will require more work. 



In [ ]:
%%writefile SimpleDynamicScheduling.java
class SimpleDynamicScheduling {

    static void sleepALittle(int numMillis) {
        try { 
            Thread.sleep(numMillis); 
        } catch(InterruptedException e) {
            // do nothing
        }
    }

    public static void main(String[] args) {
        int numThreads = Pyjama.omp_get_num_procs();
        if (args.length >= 1) {
            numThreads = Integer.parseInt(args[0]);
        }

        long startTime = System.currentTimeMillis();
        int count = 1;

        //#omp parallel for num_threads(numThreads) /* schedule(dynamic)  */
        for(int i = 1; i <= 100; i++) {
            sleepALittle(i);
        }
    
        long endTime = System.currentTimeMillis();        
        System.out.println("Time = " + (endTime-startTime) + " ms");

    }
}

In [ ]:
!java -jar lib/Pyjama.jar SimpleDynamicScheduling.java

In [ ]:
!java -cp lib/Pyjama.jar:. SimpleDynamicScheduling 4

To employ a dynamic scheduling policy, you can specify `schedule(dynamic)` or `schedule(dynamic, chunkSize)` instead of `schedule(static)` or `schedule(static, chunkSize)`. Try specifying dynamic scheduling and different chunk size and see what happens to the program's runtime.



# Parallel Sum

Very often, loops are used with an accumulator variable to compute a a single value from a set of values, such as the sum of integers in an array or list. Fortunately, OpenMP implements a special parallel pattern known as *reduction*, which will aid us in this process. The reduction pattern is one of a group of patterns called **collective communication** patterns because the threads must somehow work together to create the final desired single value.


## Initial Attempt

Let's try implementing parallel sum without using the OpenMP's _reduction_ directive. 

The `Reduction0` program below will create an array with 1 million integers that are generated randomly in the [0-1000) range. Note that `sequentialSum` sums up the array sequentially. The code in `parallelSum` uses OpenMP's parallel and for directives to attempt to sum up the array in parallel. Examine the two functions carefully. 

In [ ]:
%%writefile Reduction0.java
import java.util.Random;

class Reduction0 {
    final static int SIZE=1000000;

    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        // Generate SIZE random values in [0..1000) range
        // Fix the seed to 123 so the set of numbers will be the same in each run.
        int[] array = new Random(123).ints(SIZE, 0, 1000).toArray();
        
        System.out.println("Seq. sum: \t" + sequentialSum(array));
        System.out.println("Par. sum: \t" + parallelSum(array));
    } 


    /* sum the array sequentially */
    static int sequentialSum(int[] a) {
        int sum = 0;
        for (int i = 0; i < a.length; i++) {
            sum += a[i];
        }
        return sum;
    }

    /* sum the array using multiple threads */
    static int parallelSum(int[] a) {
        int sum = 0;
        //#omp parallel shared(a,sum) 
        {
            //#omp for 
            for(int i = 0; i < a.length;i++) {
                sum += a[i];
            }
        }
        return sum;
    }
}

In [ ]:
!java -jar lib/Pyjama.jar Reduction0.java

In [ ]:
!java -cp .:lib/Pyjama.jar Reduction0

Note that the sum from the sequentialSum is different from the sum from the parallelSum. Try running the program multiple times. 

Note that the set of random numbers generated are the same each time you run the program since we supplied the seed for the random number generator to be `123`.  

Are the results of the `sequentialSum` function changing? How about the results of the `parallelSum` function? Can you explain why the results of sequential and parallel sum are different?

How many threads were used to calculate the sum in `parallelSum`? What happens when you change this?

## Fixing the parallelSum function

Let's try a different version of the parallelSum function (below). Note that this version use the OpenMP `reduction` clause. 

The notion of a reduction comes from the mathematical operation _reduce_, in which a collection of values are combined into a single value via a common mathematical function. Summing up a collection of values is a natural example of reduction. 

In the modified program, the `reduction(+:sum)` clause tells Pyjama that a summation is being performed (the `+` operator) on the variable `sum`, which acts as the accumulator.


In [ ]:
%%writefile Reduction1.java
import java.util.Random;

class Reduction1 {
    final static int SIZE=1000000;

    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        // Generate SIZE random values in [0..1000) range
        // Fix the seed to 123 so the set of numbers will be the same in each run.
        int[] array = new Random(123).ints(SIZE, 0, 1000).toArray();

        System.out.println("Seq. sum: \t" + sequentialSum(array));
        System.out.println("Par. sum: \t" + parallelSum(array));
    } 


    /* sum the array sequentially */
    static int sequentialSum(int[] a) {
        int sum = 0;
        for (int i = 0; i < a.length; i++) {
            sum += a[i];
        }
        return sum;
    }

    /* sum the array using multiple threads */
    static int parallelSum(int[] a) {
        int sum = 0;
        //#omp parallel shared(a,sum) 
        {
            //#omp for reduction(+:sum)
            for(int i = 0; i < a.length;i++) {
                sum += a[i];
            }
        }
        return sum;
    }
}

In [ ]:
!java -jar lib/Pyjama.jar Reduction1.java

In [ ]:
!java -cp .:lib/Pyjama.jar Reduction1

Try running the program a few times and compare the results of `sequentialSum` and `parallelSum` again. Are they the same? 



## Something to ponder

What do you think happen when you specify the `reduction` clause? 

Can you fix `parallelSum` function so that it will produce the correct result without using the `reduction` clause?



The sum example that we experimented with was a fairly small example. We will look at larger examples in the following sections. 

# Integration using the Trapezoidal Rule

## Estimating the Area Under a Curve

As our next example, let’s look at the problem of estimating the area under a curve. If you have taken a calculus course, you may recognize this problem as the Riemann sum or Trapezoidal Rule, which approximates the area under the curve (i.e. the integral) by splitting the area under the curve into a series of trapezoids.

In the following programs, we attempt to use the trapezoid rule to approximate the integral:

  $\int_0^{\pi} sin(x)_{dx}$

using $2^{20}$ equal subdivisions. The answer from this computation should be 2.0. This [video](https://d32ogoqmya1dw8.cloudfront.net/files/csinparallel/workshops/numerical_integration_1_thread.mov) shows how a single thread would solve this problem. 


In the following program, a single thread serially computes the area of each trapezoid and adds all the trapezoids together into one value. A Java implementation of this program may look like the following:

In [ ]:
%%writefile Integration.java
class Integration {
    static double f(double x) {
        return Math.sin(x);
    }
    
    public static void main(String[] args) {
        //Variables

        double a = 0.0, b = Math.PI;         //limits of integration
        int n = 1048576;                //number of subdivisions = 2^20
        double h = (b - a) / n;         //width of each subdivision
        double integral;                // accumulates answer

        integral = (f(a) + f(b))/2.0;  //initial value

        //sum up all the trapezoids
        for(int i = 1; i < n; i++) {
            integral += f(a+i*h);
        }

        integral = integral * h;
        System.out.printf("With %d trapezoids, our estimate of the integral from \n", n);
        System.out.printf("%f to %f is %f\n", a,b,integral);
    } 
}

Let's compile and run this as a normal Java program:

In [ ]:
!javac Integration.java
!java Integration

## Parallel Integration - First Attempt

Let’s now consider how we can use multiple threads to approximate the area under the curve in parallel. One strategy would be to assign each thread a subset of the total set of subdivisions, so that each thread separately computes its assigned set of trapezoids.

This [video](https://d32ogoqmya1dw8.cloudfront.net/files/csinparallel/workshops/numerical_integration_4_threads.mov) illustrates how 4 threads would work together to approximate the area under the curve.

Here's an attempt to parallelize the serial version above using OpenMP and Pyjama:

In [ ]:
%%writefile Integration1.java
class Integration1 {
    static double f(double x) {
        return Math.sin(x);
    }
    
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        //Variables

        double a = 0.0, b = Math.PI;         //limits of integration
        int n = 1048576;                //number of subdivisions = 2^20
        double h = (b - a) / n;         //width of each subdivision
        double integral;                // accumulates answer

        integral = (f(a) + f(b))/2.0;  //initial value

        //sum up all the trapezoids

        //#omp parallel for shared(a, h, n, integral)
        for(int i = 1; i < n; i++) {
            integral += f(a+i*h);
        }

        integral = integral * h;
        System.out.printf("With %d trapezoids, our estimate of the integral from \n", n);
        System.out.printf("%f to %f is %f\n", a,b,integral);
    } 
}

In [ ]:
!java -jar lib/Pyjama.jar Integration1.java

If our parallel program is implemented correctly, the program should estimate the area under the curve as 2.00, which would be identical to the output of the serial program.

Try running the program with different number of threads: 4, 2, 1. When will it produce the correct/expected value (2.0)?

In [ ]:
!java -cp .:lib/Pyjama.jar Integration1 4

In [ ]:
!java -cp .:lib/Pyjama.jar Integration1 2

In [ ]:
!java -cp .:lib/Pyjama.jar Integration1 1

## Parallel Integration - Second Attempt

Now, let's use the previously discussed `reduction` clause to fix the issue that we observed in previous section. Here's the improved version below.

In [ ]:
%%writefile Integration2.java
class Integration2 {
    static double f(double x) {
        return Math.sin(x);
    }
    
    public static void main(String[] args) {
        if (args.length >= 1) {
            Pyjama.omp_set_num_threads(Integer.parseInt(args[0]));
        }
        System.out.println();

        //Variables

        double a = 0.0, b = Math.PI;         //limits of integration
        int n = 1048576;                //number of subdivisions = 2^20
        double h = (b - a) / n;         //width of each subdivision
        double integral;                // accumulates answer

        integral = (f(a) + f(b))/2.0;  //initial value

        //sum up all the trapezoids

        //#omp parallel for shared(a, h, n) reduction(+:integral)
        for(int i = 1; i < n; i++) {
            integral += f(a+i*h);
        }

        integral = integral * h;
        System.out.printf("With %d trapezoids, our estimate of the integral from \n", n);
        System.out.printf("%f to %f is %f\n", a,b,integral);
    } 
}

As with the previous reduction patternlet example, we again have an accumulator variable in the loop, this time called integral. Thus our reduction clause reads reduction(+:integral).

Compile and run the above program with 4 threads using the following commands:

In [ ]:
!java -jar lib/Pyjama.jar Integration2.java

In [ ]:
!java -cp .:lib/Pyjama.jar Integration2 4

Did you get the expected result? (2.0)

**Note**: 

Because we did not explicitly add an additional static or dynamic clause to the pragma on line 24 on the working version above, the default behavior of assigning equal amounts of work doing consecutive iterations of the loop (i.e. static scheduling) was used to decompose the problem onto threads. In this case, since the number of trapezoids used was 1048576, then with 4 threads, thread 0 will do the first 1048576/4 trapezoids, thread 1 the next 1048576/4 trapezoids, and so on.

# Performance Study - Drug Design Exemplar

Let’s look at a larger example. An important problem in Biology is that of drug design. The goal is to find small molecules, called _ligands_, that are good candidates for use as drugs.

This is a very rough simulation of a program to compute how well a set of short protein ligands (each a possible drug) matches a given longer protein string. In the real software programs that do this, the matching is quite sophisticated, targeting possible "receptor" sites on the protein.

Here is an image illustrating the concept of the ligand (represented by small sticks in center) binding to areas of the protein (represented by ribbon structure):


<img src="https://pdcbook.calvin.edu/pdcbook/RaspberryPiHandout/_images/proteinligand.jpg">

For the real versions of this code and our simulated case, the longer the ligand or the longer the protein, the longer it takes for the matching and score of the match to complete.

We have created a default fake protein in the code. This can be changed on the command line.

We create the list of possible ligands by choosing random lengths (controlled by the `maxligand` variable). Thus, each ligand can be of a different length.

## The Drug Design Exemplar

Unlike previous code examples, our primary focus this section is analyzing the performance of the static and dynamic OpenMP programs.

An implementation of the exemplar is below. Note that in addition to changing the number of threads, you should also try different scheduling strategy (static vs dynamic). 

In [ ]:
%%writefile DDPyjama.java
import java.util.Random;

public class DDPyjama {
    static Random rand = new Random(42);

    static String[] cannedLigands = 
        {"razvex", "qudgy", "afrs", "sst", "pgfht", "rt", 
        "id", "how", "aaddh",  "df", "os", "hid", 
        "sad", "fl", "rd", "edp", "dfgt", "spa"};

    // Ligand Score pair
    static class LSPair {
        String ligand;
        int score;
    
        public LSPair(String ligand, int score) {
            this.ligand = ligand;
            this.score = score;
        }

        @Override
        public String toString() {
            return "["+ligand+","+score+"]";
        }
    }

    // returns arbitrary string of lower-case letters of length at most max_ligand
    static String makeLigand(int maxLigandLength) {

        int len = rand.nextInt(maxLigandLength+1);
        
        StringBuilder sb = new StringBuilder();
        for (int i = 0;  i < len;  i++) 
            sb.append((char) ('a' + rand.nextInt(26)));  
        return sb.toString();
    }
  
    private static String[] generateLigands(int numLigands, int maxLigandLength, boolean useCanned) {
        // If we use canned ligands, use as many of them as we can, then fill the rest with randomly generated ligands. 
        // Otherwise, create a set of ligands whose length randomly varies from 1 to args.maxLigand

        String[] result = new String[numLigands];

        if (useCanned) {
            for(int i = 0; i < Math.min(numLigands, cannedLigands.length); i++) {
                result[i] = cannedLigands[i];
            }
        }

        for(int i = useCanned ? cannedLigands.length : 0; i < numLigands; i++) {
            result[i] = makeLigand(maxLigandLength);
        }
        return result;
    }

    public static void main(String[] args) {

        if (args.length != 4) {
            System.out.println("Usage DDPyjama numThreads numLigands maxLigandLength protein useCanned printLigands");

            // the example string below is one of Dijkstra's famous quotes
            System.out.println("   Example: java -cp .:Pyjama.jar DDPyjama 4 10 8 \"Simplicity is a great virtue but it requires hard work to achieve it and education to appreciate it\" false true\n");
        }

        int numThreads = 4;
        if (args.length >= 1) {
            numThreads = Integer.parseInt(args[0]);
        }

        int numLigands = 12;
        if (args.length >= 2) {
            numLigands = Integer.parseInt(args[1]);
        }

        int maxLigandLength = 6;
        if (args.length >= 3) {
            maxLigandLength = Integer.parseInt(args[2]);
        }

        String protein = "the cat in the hat wore the hat to the cat hat party";

        if (args.length >= 4) {
            protein = args[3];
        }

        System.out.println("Number of threads: " + numThreads);
        System.out.println("Number of ligands: "+numLigands);
        System.out.println("Max ligand length: "+ maxLigandLength);
        System.out.println("Protein: "+ protein);
        System.out.println();

        // Things to do: 
        // 1. Generate the requested numLigands w/ maxLigandLength
        // 2. Calculate the matching score for each ligand vs the given protein
        //    Score is calculated based on the number of character in the ligand that
        //    appears in the same order in the protein. 
        // 3. Find the ligand(s) with the highest score

        long start = System.currentTimeMillis();
        String[] ligands = generateLigands(numLigands, maxLigandLength, args.length >= 5 && args[4].equals("true"));

        // print the ligands if desired
        if (args.length >= 6 && args[5].equals("true")) {
            System.out.println("Here are the ligands");
            for(String l : ligands) {
                System.out.println(l);
            }
        }
        
        // map each ligand to (ligand, score)
        // also keep track of the maxScore 
        LSPair[] ligandsWScore = new LSPair[numLigands];
        int maxScore = 0;

        //#omp parallel for num_threads(numThreads) shared(numLigands, ligands, ligandsWScore, protein) reduction(max:maxScore) schedule(dynamic)
        for(int i = 0; i < numLigands; i++) {
            String ligand = ligands[i];
            int score = calcScore(ligand, protein);
            ligandsWScore[i] = new LSPair(ligand, score);
            maxScore = Math.max(maxScore, score);
        }

        // find the ligands whose score is maxScore
        // this is a reduce operation
        StringBuilder sb = new StringBuilder();
        for(int i = 0; i < numLigands; i++) {
            if (ligandsWScore[i].score == maxScore) {
                if (sb.length() > 0) sb.append(", ");
                sb.append(ligandsWScore[i].ligand);
            }
        }

        long end = System.currentTimeMillis();
        System.out.println("The maximum score is " + maxScore);
        System.out.println("Achieved by ligand(s) "+ sb.toString());
        System.out.println("Calculation time " + (end-start) + " ms");
    }

    /**
     * Match a ligand (str1) and the protein. Count the number of characters in str1
     * that appear in the same seq in str2 (there can be any number of intervening chars)
     * @param str1 first string
     * @param str2 second string
     * @return number of matches
     */
    private static int calcScore(String str1, String str2) {
        // no match if either is empty string
        if (str1.length() == 0 || str2.length() == 0) return 0;

        if (str1.charAt(0) == str2.charAt(0)) {
            return 1 + calcScore(str1.substring(1), str2.substring(1));
        }
        return Math.max(
            calcScore(str1, str2.substring(1)), calcScore(str1.substring(1), str2));
    }
}

In [ ]:
!java -jar lib/Pyjama.jar DDPyjama.java

In [ ]:
!java -cp .:lib/Pyjama.jar DDPyjama

Try running it with more ligands and longer ligands length. Observe how much longer it takes for the program to finish.

In [ ]:
!java -cp .:lib/Pyjama.jar DDPyjama 4 20 6 "your time is limited, so don't waste it living someone else's life"

## Speedup

**Speedup** is the ratio of the execution time of a serial program to its parallel execution. Specifically,

  $S_n= \frac{T_1}{T_n}$

Where $T_1$ is the time it takes to execute a program on 1 thread, while $T_n$ is the time it takes to execute a program on n threads. Some important notes:

 - A speedup greater than 1 indicates that there is benefit to the parallel implementation. A speedup of x means that the parallel code is x times faster.
 - A speedup less than 1 indicates that there is no benefit to parallelism.

To correctly calculate speedup, we must first measure the running time of a program. In this case, we will use `System.currentTimeMillis` to measure how long it takes for the program to finish, starting from the ligand generation step.



## Performance study

 Try running the program using different number of threads and different scheduling strategy and complete the table below.

Time (s) | 1 Thread | 2 Threads | 3 Threads | 4 Threads
---------|----------|-----------|-----------|----------
dynamic  |          |           |           |
static   |          |           |           |


How does the run time of dynamic vs static scheduling compare when the program is run with multiple threads?

We are using Python to assisst us with the speed up calculation below. 

Try calculate the speed-up for each of the (thread number, scheduling) combination. Replace the `??` in the Python code below with numbers from your table. 

In [ ]:
#lists holding measured times (floating point)
#TODO: Fill in arrays below (code will not compile otherwise!)
#Time (n threads)  1   2  3  4
dd_static =       [ ??, ??, ??, ??]
dd_dynamic=       [ ??, ??, ??, ??]

#compute speedup
static_speedup  = [round(dd_static[0]/dd_static[i],2)   for i in range(1,4)]
dynamic_speedup = [round(dd_dynamic[0]/dd_dynamic[i],2) for i in range(1,4)]

#print results
print("static speedup:")
print(static_speedup)

print("dynamic speedup:")
print(dynamic_speedup)

## Summary - Static vs Dynamic Scheduling

In many cases, static scheduling is sufficient. However, there is an implicit assumption with static scheduling that all components take about the same amount of time. However, if some components take longer than others, a **load balancing** issue can arise. In the case of the drug design example, different ligands take longer to compute than others. Therefore, a dynamic scheduling approach is better.

Next, note that speedup is non-linear, especially at higher number of threads. While increasing the number of threads is supposed to generally _improve_ the run time of a program, it usually does not scale linearly with the number of cores. This occurs for a number of reasons. First, at higher number of threads, it is more likely that the serial components of a program (such as thread creation overhead, and any other necessary serial operations that must take place before the parallel code can run) start dominating run time (see **Amdahl’s Law**). Second, **resource contention** on the CPU or memory from other processes can cause slow downs. A related measure called **efficiency** measures how _well_ a program uses the cores assigned to it.

Further Reading on Performance: [Dive into Systems, Chapter 14.4](https://diveintosystems.org/antora/diveintosystems/1.0/SharedMemory/performance.html)